In [2]:
import pandas as pd 
import pyodbc

C:\Users\Admin\AppData\Local\Temp\ipykernel_19264\3183896952.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## **1\. User experience comparison**

**Main problem**: Identification for user experience's change between 2 versions.

**Hypothesis**<span style="background-color: rgb(43, 43, 43); color: var(--vscode-foreground);">: With the given information of bad user experience in the tutorial and my own experience after several test runs, it is apparent that the tutorial section failed to explain the game's rule. I theorized that with the uncertanty of not knowing how to play the game, a good amount of players would struggle at level 1 eventhough it is supposed to be the easiest level but. To be able to prove this, I compiled a query to first acknowledging the tutorial completion rate.&nbsp;</span>

In [4]:
SELECT 
    nop.game_vers,
    nop.player as num_of_player,
    notc.player as tut_completed,
    (CAST(notc.player as FLOAT) / CAST(nop.player as FLOAT)) *100 
        as tut_complete_percentage
FROM (
    SELECT 
        game_vers,
        COUNT(DISTINCT player) as player 
    FROM 
        IndieZ_160..pixel_art
    GROUP BY 
        game_vers
) as nop
JOIN (
    SELECT 
        game_vers,
        COUNT(DISTINCT player) as player 
    FROM 
        IndieZ_160..pixel_art
    WHERE 
        quantity = -2 
    GROUP BY 
        game_vers
) as notc
ON nop.game_vers = notc.game_vers 


IndentationError: unexpected indent (609922207.py, line 2)

The tutorial completion rate is fairly similar across both versions. Then using this information, I complied another query to get the number of players that completed the tutorial but lost at level 1:

In [3]:
SELECT 
    tut_complete.game_vers,
    tut_complete.player as tut_completed_player,
    lv_1_lost.player as lv_1_lost_player,
    ROUND(CAST(lv_1_lost.player as FLOAT) / CAST(tut_complete.player as FLOAT) * 100, 2) as percentage
FROM (
    SELECT 
        game_vers,
        COUNT(DISTINCT player) as player 
    FROM 
        IndieZ_160..pixel_art
    WHERE 
        quantity = -2 
    GROUP BY 
        game_vers
) as tut_complete
JOIN (
    SELECT 
        game_vers,
        COUNT(DISTINCT player) as player
    FROM 
        IndieZ_160..pixel_art
    WHERE 
        player IN (
            SELECT  
                player
            FROM 
                IndieZ_160..pixel_art
            WHERE 
                event_name = 'tutorial' AND quantity = -2) 
        AND level = 1 AND win = 0
    GROUP BY 
        game_vers
) as lv_1_lost
ON tut_complete.game_vers = lv_1_lost.game_vers 

(2 rows affected)

Total execution time: 00:00:00.492

game_vers,tut_completed_player,lv_1_lost_player,percentage
1.5.2,6341,1561,24.62
1.6.0,6571,1373,20.89


Using the same logic as the corellation between the uncertanty of the game's rule and level 1's losing rate, I hypothesized that the number of loses would drop significantly in level 2 since most player would understand how the game works by that time:

In [4]:
SELECT 
    tut_complete.game_vers,
    tut_complete.player as num_of_lv_2_player,
    lost.player as lv_2_lost_player,
    ROUND(CAST(lost.player as FLOAT) / CAST(tut_complete.player as FLOAT) * 100, 2) as percentage
FROM (
    SELECT 
        game_vers,
        COUNT(DISTINCT player) as player 
    FROM 
        IndieZ_160..pixel_art
    WHERE 
        level = 2
    GROUP BY 
        game_vers
) as tut_complete
JOIN (
    SELECT 
        game_vers,
        COUNT(DISTINCT player) as player
    FROM 
        IndieZ_160..pixel_art
    WHERE 
        player IN (
            SELECT  
                player
            FROM 
                IndieZ_160..pixel_art
            WHERE 
                event_name = 'tutorial' AND quantity = -2) 
            AND 
                level = 2 AND win = 0
    GROUP BY 
        game_vers
) as lost
ON tut_complete.game_vers = lost.game_vers 

(2 rows affected)

Total execution time: 00:00:00.462

game_vers,num_of_lv_2_player,lv_2_lost_player,percentage
1.5.2,6223,56,0.9
1.6.0,6396,48,0.75


As expected, the lost rate dropped dramatically from more than 20% to less than 1% in both versions.